In [2]:
from __future__ import division, print_function, absolute_import

import collections
import os
import random
import urllib
import zipfile

import numpy as np
import tensorflow as tf

In [3]:
#Traning Parameters
learning_rate = 0.1
batch_size = 128
num_steps = 3000000
display_step = 10000
eval_step = 200000

#Evaluation Parameters
eval_words = ['five', 'of', 'going', 'hardware', 'american', 'britain']

#Word2Vec Parameters
embedding_size = 200 #Dimension of the embedding vector
max_vocabulary_size = 50000
min_occurrence = 10 #Remove words that do not appear at least n times
skip_window = 3 # How many words to consider (left and right of a center word)
num_skips = 2 # How many times to reuse an input to generate a label
num_sampled = 64 # Num of negative examples to sample

In [6]:
#Download a small chunk of Wikipedia articles collection
url = 'http://mattmahoney.net/dc/text8.zip'
data_path = 'text8.zip'
if not os.path.exists(data_path):
    print("Downloading the dataset... It takes time...")
    filename, _ = urllib.request.urlretrieve(url, data_path)
    print("Finished!")
#Unzip the dataset file,Text has already been processed
with zipfile.ZipFile(data_path) as f:
    text_words = f.read(f.namelist()[0]).lower().split()

Finished!


In [68]:
for i, word in enumerate(text_words):
    word = word.decode('UTF-8')
    text_words[i] = word

In [72]:
#Build the dictionary and replace rare words with UNK token
count = [("UNK", -1)]
#Retrieve the most common words
count.extend(collections.Counter(text_words).most_common(max_vocabulary_size - 1))
#Remove samples with less than min_occurence occurrences
for i in range(len(count) - 1, -1, -1):
    if count[i][1] < min_occurrence:
        count.pop(i)
    else:
        break # -1 order, once not <, don't need to retrieve any more.
#Computer the vovabulary size
vocabulary_size = len(count)
#Assign id to each word
word2id = dict()
for i, (word, _) in enumerate(count):
    word2id[word] = i

data = list()
unk_count = 0
#Retrieve word id, or assign it index 0 ("UNK") if not in dict
for word in text_words:
    index = word2id.get(word, 0)
    if index == 0:
        unk_count += 1
    data.append(index)
count[0] = ("UNK", unk_count)
id2word = dict(zip(word2id.values(), word2id.keys()))

print("Words count:\n", len(text_words))
print("Most Common Words:\n", count[:10])
print("Vocabulary size:\n", vocabulary_size)
print("Unique words:\n", len(set(text_words)))

Words count:
 17005207
Most Common Words:
 [('UNK', 444176), ('the', 1061396), ('of', 593677), ('and', 416629), ('one', 411764), ('in', 372201), ('a', 325873), ('to', 316376), ('zero', 264975), ('nine', 250430)]
Vocabulary size:
 47135
Unique words:
 253854


In [73]:
data_index = 0
# Generate training batch for the skip-gram model
def next_batch(batch_size, num_skips, skip_window):
    global data_index
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape = (batch_size), dtype = np.int32)
    labels = np.ndarray(shape = (batch_size, 1), dtype = np.int32)
    #Windows' size
    span = 2 * skip_window + 1 # 1 represents the center word
    buffer = collections.deque(maxlen = span)
    if data_index + span > len(data):
        data_index = 0
    buffer.extend(data[data_index : data_index + span])
    data_index += span
    for i in range(batch_size // num_skips):
        context_word = [w for w in range(span) if w != skip_window]
        words_to_use = random.sample(context_word, num_skips)
        for j, context_word in enumerate(words_to_use):
            batch[i * num_skips + j] = buffer[skip_window]
            labels[i * num_skips + j, 0] = buffer[context_word]
        if data_index == len(data):
            buffer.extend(data[0 : span])
            data_index = span
        else:
            buffer.append(data[data_index])
            data_index += 1
    data_index = (data_index + len(data) - span) % len(data)
    return batch, labels

In [74]:
#Input
X = tf.placeholder(tf.int32, shape = [None])
Y = tf.placeholder(tf.int32, shape = [None, 1])

with tf.device('/cpu:0'):
    embedding = tf.Variable(tf.random_normal([vocabulary_size, embedding_size]))
    # Lookup the corresponding embedding vectors for each sample in X
    X_embed = tf.nn.embedding_lookup(embedding, X)
    # NCE Loss Variables
    nce_weights = tf.Variable(tf.random_normal([vocabulary_size, embedding_size]))
    nce_bias = tf.Variable(tf.random_normal([vocabulary_size]))
    
#Compute the average NCE loss for the batch
loss_op = tf.reduce_mean(
    tf.nn.nce_loss(weights = nce_weights,
                  biases = nce_bias,
                  labels = Y,
                  inputs = X_embed,
                  num_sampled = num_sampled,
                  num_classes = vocabulary_size))

#Optimizer 
optimizer = tf.train.GradientDescentOptimizer(learning_rate)
train_op = optimizer.minimize(loss_op)

#Evaluation
#Compute the cosine similarity between input data embedding and every embedding vectors
X_embed_norm = X_embed / tf.sqrt(tf.reduce_sum(tf.square(X_embed)))
embedding_norm = embedding / tf.sqrt(tf.reduce_sum(tf.square(embedding), 1, keepdims = True))
cosine_sim_op = tf.matmul(X_embed_norm, embedding_norm, transpose_b = True)

In [ ]:
#Initialize the variables
init = tf.global_variables_initializer()

with tf.Session() as sess:
    
    sess.run(init)
    
    x_test = np.array([word2id[w] for w in eval_words])
    
    average_loss = 0
    for step in range(1, num_steps + 1):
        batch_x, batch_y = next_batch(batch_size, num_skips, skip_window)
        _, loss = sess.run([train_op, loss_op], feed_dict = {X : batch_x, Y : batch_y})
        average_loss += loss
        
        if step % display_step == 0 or step == 1:
            if step > 1:
                average_loss /= display_step
            print("Step " + str(step) + ", Average Loss = " + \
                 "{:4f}".format(average_loss))
            average_loss = 0
            
        #Evaluatiom
        if step % eval_step == 0 or step == 1:
            print("Evaluation...")
            sim = sess.run(cosine_sim_op, feed_dict = {X : x_test})
            for i in range(len(eval_words)):
                top_k = 8 # num of nearest neighbors
                nearest = (-sim[i, :]).argsort()[1:top_k + 1]
                log_str = '"%s" nearest neighbors:' % eval_words[i]
                for k in range(top_k):
                    log_str = '%s %s' % (log_str, id2word[nearest[k]])
                    print(log_str)

Step 1, Average Loss = 515.931152
Evaluation...
"five" nearest neighbors: paymaster
"five" nearest neighbors: paymaster guarantee
"five" nearest neighbors: paymaster guarantee sultanate
"five" nearest neighbors: paymaster guarantee sultanate physik
"five" nearest neighbors: paymaster guarantee sultanate physik despise
"five" nearest neighbors: paymaster guarantee sultanate physik despise plug
"five" nearest neighbors: paymaster guarantee sultanate physik despise plug awe
"five" nearest neighbors: paymaster guarantee sultanate physik despise plug awe reconciled
"of" nearest neighbors: tauris
"of" nearest neighbors: tauris lleida
"of" nearest neighbors: tauris lleida caryophyllales
"of" nearest neighbors: tauris lleida caryophyllales wollheim
"of" nearest neighbors: tauris lleida caryophyllales wollheim rear
"of" nearest neighbors: tauris lleida caryophyllales wollheim rear billy
"of" nearest neighbors: tauris lleida caryophyllales wollheim rear billy solubility
"of" nearest neighbors: t

Step 410000, Average Loss = 9.815817
Step 420000, Average Loss = 9.278574
Step 430000, Average Loss = 9.504482
Step 440000, Average Loss = 9.210628
Step 450000, Average Loss = 9.262757
Step 460000, Average Loss = 8.928328
Step 470000, Average Loss = 8.622349
Step 480000, Average Loss = 8.793839
Step 490000, Average Loss = 9.031920
Step 500000, Average Loss = 8.872289
Step 510000, Average Loss = 8.563657
Step 520000, Average Loss = 8.632819
Step 530000, Average Loss = 8.650152
Step 540000, Average Loss = 8.376483
Step 550000, Average Loss = 8.212923
Step 560000, Average Loss = 8.200018
Step 570000, Average Loss = 8.135792
Step 580000, Average Loss = 7.999315
Step 590000, Average Loss = 8.090540
Step 600000, Average Loss = 7.854766
Evaluation...
"five" nearest neighbors: six
"five" nearest neighbors: six four
"five" nearest neighbors: six four three
"five" nearest neighbors: six four three two
"five" nearest neighbors: six four three two seven
"five" nearest neighbors: six four three two

Step 1010000, Average Loss = 6.619600
Step 1020000, Average Loss = 6.830632
Step 1030000, Average Loss = 6.686428
Step 1040000, Average Loss = 6.675690
Step 1050000, Average Loss = 6.633317
Step 1060000, Average Loss = 6.666533
Step 1070000, Average Loss = 6.675185
Step 1080000, Average Loss = 6.529944
Step 1090000, Average Loss = 6.555566
Step 1100000, Average Loss = 6.631552
Step 1110000, Average Loss = 6.471942
Step 1120000, Average Loss = 6.569354
Step 1130000, Average Loss = 6.462279
Step 1140000, Average Loss = 6.543951
Step 1150000, Average Loss = 6.644977
Step 1160000, Average Loss = 6.501319
Step 1170000, Average Loss = 6.539650
Step 1180000, Average Loss = 6.442169
Step 1190000, Average Loss = 6.480172
Step 1200000, Average Loss = 6.412401
Evaluation...
"five" nearest neighbors: four
"five" nearest neighbors: four six
"five" nearest neighbors: four six three
"five" nearest neighbors: four six three seven
"five" nearest neighbors: four six three seven eight
"five" nearest neig

Step 1610000, Average Loss = 6.054028
Step 1620000, Average Loss = 6.010270
Step 1630000, Average Loss = 6.105283
Step 1640000, Average Loss = 6.018218
Step 1650000, Average Loss = 6.082092
Step 1660000, Average Loss = 5.990065
Step 1670000, Average Loss = 6.068949
Step 1680000, Average Loss = 6.131071
Step 1690000, Average Loss = 6.084009
Step 1700000, Average Loss = 6.049506
Step 1710000, Average Loss = 6.004802
Step 1720000, Average Loss = 6.032802
Step 1730000, Average Loss = 5.973735
Step 1740000, Average Loss = 6.088704
Step 1750000, Average Loss = 5.945672
Step 1760000, Average Loss = 6.033279
Step 1770000, Average Loss = 5.984999
Step 1780000, Average Loss = 5.979765
Step 1790000, Average Loss = 5.941413
Step 1800000, Average Loss = 5.706453
Evaluation...
"five" nearest neighbors: four
"five" nearest neighbors: four six
"five" nearest neighbors: four six three
"five" nearest neighbors: four six three seven
"five" nearest neighbors: four six three seven eight
"five" nearest neig

Step 2210000, Average Loss = 5.859318
Step 2220000, Average Loss = 5.838483
Step 2230000, Average Loss = 5.795220
Step 2240000, Average Loss = 5.819814
Step 2250000, Average Loss = 5.770585
Step 2260000, Average Loss = 5.762588
Step 2270000, Average Loss = 5.806811
Step 2280000, Average Loss = 5.712869
Step 2290000, Average Loss = 5.828200
Step 2300000, Average Loss = 5.755219
Step 2310000, Average Loss = 5.806276
Step 2320000, Average Loss = 5.697569
Step 2330000, Average Loss = 5.517050
Step 2340000, Average Loss = 5.694434
Step 2350000, Average Loss = 5.786160
Step 2360000, Average Loss = 5.764073
Step 2370000, Average Loss = 5.684353
Step 2380000, Average Loss = 5.714607
Step 2390000, Average Loss = 5.740133
Step 2400000, Average Loss = 5.744644
Evaluation...
"five" nearest neighbors: four
"five" nearest neighbors: four three
"five" nearest neighbors: four three six
"five" nearest neighbors: four three six two
"five" nearest neighbors: four three six two seven
"five" nearest neighb